--------------------------------------------------------------------------------------

* Team member names: Elsie Wang, Hou Wan

* Team member IDs: A16655032, A16462095

--------------------------------------------------------------------------------------

# Optimizing Camping Spots with OpenAI and ArcGIS

## Introduction

***TODO***

Question(s) you addressed, why it is important (5pt)

As in your project proposal, describe the intended audience and the business case for finding a
solution to your question.

If your question changed significantly compared to your project proposal, please explain.
(format this as a markdown cell)

(at least 250 words):


### Related Work

***TODO***

At least 4 bibliographic references (with links) relevant to your question. These don’t have to be
new references; you can continue what you started in your project proposal. As before, the
references can include research publications, online articles, blogs, github repos, online
applications, etc.

As a new element of this section, reflect on how these references helped you specify the
problem more narrowly, or focus on a specific hypothesis, or question conclusions they made,
or improve over previously available code.
(format this as a markdown cell)

(at least 200 words)

### Python Packages

***TODO***

Make this a brief list – but also mention whether and how this list has evolved compared to the
project proposal.

In your notebook: identify and describe the libraries as you import them, using a combination
of markdown and code cells.

(at least 100 words)

### References

***TODO***

Include a URL and a one-sentence description of each source you used. Make sure you have
enough records for your machine learning application.

Please reflect on how your choice of sources evolved since the proposal phase, and any
concerns about the sources you used - related to data quality, provenance, access constraints,
etc. Also, reflect on any data that would be helpful to address your research question, which
you could not obtain (and why). Be creative! There is a lot of additional information, sometimes
from less traditional sources, that may help.

When you create a notebook, identify and describe the datasets as you load them. If the
sources are in AGOL, provide source IDs, and/or load them using gis.content.get. Provide
additional discussion in a markdown cell.

(at least 200 words)


--------------------------------------------------------------------------------------

In [58]:
# Import statements, etc.
import warnings
warnings.filterwarnings("ignore")

from json import load
import os
import pandas as pd
import geopandas as gpd
import numpy as np

import arcgis
from arcgis.gis import GIS
from arcgis.raster.functions import *
from ipywidgets import *
from IPython.display import display
from arcgis.geocoding import geocode
from arcgis.geometry import *

# Used with github repo
LOGIN = load(open('./login.json', 'r'))
gis = GIS(username=LOGIN['username'], password=LOGIN['password'])

# TODO: Replace above with this in final notebook
# import getpass
# username = input('Enter username: ')
# password = getpass.getpass("Enter password: ")
# gis = GIS(username=username, password=password)

## Data Cleaning

***TODO***

Describe the cleaning/wrangling operations that you performed, and whether you realized you
needed to clean the data by examining metadata documents or by introspecting the data itself.
Was the amount of cleaning/data preparation similar to what you expected when writing
project proposal?

Organize the discussion and code as a combination of markdown and code cells.
(at least 100 words – less if you didn’t have to do any cleaning!)

Potential Factors:
- primitive vs campground*
- light pollution
- water accessibility
- stargazing
- proximity from roads
- proximity from hiking trails
- near water
- tent vs car vs rv camping
- weather: winter vs summer camping
- campfires
- ammentities: water, showers, restrooms, mobile service


*essential information

### Load Datasets

In [33]:
data_location = os.environ["HOME"] + '/dsc170_finalproject_camping/data/'

In [59]:
park_boundaries = gpd.read_file(data_location + 'park_boundaries/ParkBoundaries.shp')
campgrounds = gpd.read_file(data_location + 'campgrounds/Campgrounds.shp')
buildings = gpd.read_file(data_location + 'buildings/Buildings.shp')
parking = gpd.read_file(data_location + 'parking/ParkingPoints.shp')

## Descriptive Statistics

***TODO***

Explore the data using maps, charts, and common descriptive statistics. Sample questions you
can try to answer: is there spatial autocorrelation in the data? Are you dealing with random
point patterns? What is the spatial mean and standard distance? Or why the above questions
are not relevant to your research theme and the data?

(a combination of markdown and documented code cells)

### Park Boundaries

In [64]:
buildings[buildings['TYPE'].isna()]

,FID,GISID,DETAIL,TYPE,SUBCLASS,UNITNBR,UNITNAME,SHARE,GlobalID,geometry
34,2398,GIS0111090,Gin Yet Wah Cabin,None,Historic,356,Malakoff Diggins SHP,Public,{A8E16AD4-B720-4A6E-BCDC-866D34589B5E},POINT (-13459252.292 4774468.278)
36,2400,GIS0111092,Gaus Residence,None,Historic,356,Malakoff Diggins SHP,Public,{DB080BBE-055C-4CD9-8D4B-F5939463AB5B},POINT (-13458746.130 4774335.187)
37,2401,GIS0111093,Cummins Hall,None,Historic,356,Malakoff Diggins SHP,Public,{D561E37F-3099-446C-AAF0-F6C53A8EB5BA},POINT (-13458459.058 4774546.749)
38,2402,GIS0111094,Kings Saloon,None,Historic,356,Malakoff Diggins SHP,Public,{7D59067B-9AA2-49F7-97CE-87484A2955DF},POINT (-13458497.690 4774528.778)
39,2403,GIS0111095,Smith-Knotwell Drug Store,None,Historic,356,Malakoff Diggins SHP,Public,{5275E0E1-2263-4AED-A9DE-5DCC1B12F142},POINT (-13458512.755 4774509.289)
40,2404,GIS0111096,Watson Residence,None,Historic,356,Malakoff Diggins SHP,Public,{2CE54E3B-CF59-413C-BC08-8C708C293CB0},POINT (-13458427.570 4774585.159)
41,2405,GIS0111097,Landsburg Residence,None,Historic,356,Malakoff Diggins SHP,Public,{DA5D451A-5830-4D28-9F9F-6B8539D94964},POINT (-13458405.314 4774599.686)
42,2406,GIS0111098,Ostrom Residence,None,Historic,356,Malakoff Diggins SHP,Public,{FA605A69-E284-43A5-9C4A-5D7C9FBEE07C},POINT (-13458386.661 4774627.720)
43,2407,GIS0111099,Cummins Barn,None,Historic,356,Malakoff Diggins SHP,Public,{9C5A263C-1DDA-4CB0-A480-C835A0F42740},POINT (-13458388.956 4774951.932)
44,2408,GIS0111100,Blaine Residence,None,Historic,356,Malakoff Diggins SHP,Public,{B89D4E79-A12E-4902-9BEC-3C88D378FC82},POINT (-13458295.847 4775149.955)


### Campgrounds

--------------------------------------------------------------------------------------

## Analysis

***TODO***

Provide a general outline of your analysis (in a markdown), and then document it step by step
as you code the solution. Please make sure that we can reproduce your analysis by running
your notebook. As before, a diagram describing your workflow would be helpful.

Please also reflect on how your actual analysis steps were different compared to your project
proposal - or state that you didn’t deviate from the initial plan.

(a combination of markdown and documented code cells)
(at least 500 words)

--------------------------------------------------------------------------------------

## Results

***TODO***

Describe what you found, and why it is important; illustrate the findings with maps/charts
reflecting your results.

(a combination of markdown and documented code cells)

(200 words)

## Discussion

The discussion should include the following parts:
1) Discuss your findings with respect to the literature sources in section 3. What do the
results mean in the context of what is already known? What is new? Does it validate
what was found in literature? How do your results improve our understanding of the
problem?

2) Of particular importance is a discussion of any trade-offs and decision points that you
had to consider. This may include a discussion of any performance issues, width of
buffers you applied, projections you chose, spatial operations you used, machine
learning techniques, and other issues we discussed in class.
(this can be done as a markdown, at least 200 words)

## Conclusions and Future Work

Did you manage to completely answer your initial research question? If not, what additional
data and additional analysis steps can you think of? Can your approach be extended to other
areas or topics, and use additional datasets? How do you expect the results to be used and by
whom?

(a markdown cell, at least 200 words)
